In [1]:
import tensorflow as tf
import csv
import math
from tensorflow import keras



# Helper libraries
import numpy as np

import rospy
import os
import json
import numpy as np
import random
import time
import sys
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Activation
from collections import deque

In [ ]:
batch_size = 2
train_start = 64
memory = deque(maxlen=1000000)


In [ ]:

from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
state = keras.layers.Input(shape=(14,), name='state')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
d1 = Dense(512, activation='relu')(state)
#auxiliary_output = keras.layers.Dense(1, activation='sigmoid', name='aux_output')(d1)

action = keras.layers.Input(shape=(2,), name='action')
x = keras.layers.concatenate([d1, action])

x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)


# And finally we add the main logistic regression layer
q_val= Dense(1, activation='sigmoid', name='main_output')(x)

critic = Model(inputs=[state, action], outputs=[q_val])
#critic.summary()

In [ ]:


state = keras.layers.Input(shape=(14,), name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Dense(512, activation='relu')(state)
x = Dense(512, activation='relu')(state)
x = Dense(512, activation='relu')(state)
#auxiliary_output = keras.layers.Dense(1, activation='sigmoid', name='aux_output')(d1)

lin_vel = Dense(1, activation='sigmoid', name='lin_vel')(x)
ang_vel = Dense(1, activation='tanh', name='ang_vel')(x)
output = keras.layers.concatenate([lin_vel, ang_vel])

actor = Model(inputs=[state], outputs=output)
#actor.summary()


In [ ]:
import numpy as np

In [ ]:
critic.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mse',
              metrics=['accuracy'])

actor.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mse',
              metrics=['accuracy'])


In [ ]:
#model.fit(ml_in, ml_out, epochs=10)
test = np.ones(14)
test2 = np.ones(2)


In [ ]:
#predictions = model.predict(ml_in)
blank = actor.predict(test.reshape(1, len(test)))
blank2 = critic.predict([test.reshape(1, len(test)), test2.reshape(1,len(test2))])

In [ ]:
#predictions[0:20]
#predictions.shape
past_action = np.zeros((1,2))
print blank.shape
past_action.shape

In [ ]:
t = np.empty([0,14])
t2 = np.empty([0,2])
test = np.ones(14)
test2 = np.ones(2)
test = test.reshape(1, len(test))
test2 = test2.reshape(1, len(test2))
print test2.shape
print t2.shape
t = np.append(t , test, axis = 0)
t2 = np.append(t2 , test2, axis = 0)

print test.shape
print test2.shape

In [ ]:
blank2 = critic.predict([test, test2])


In [ ]:
test[0][2] = 3
print test.shape
print test2.shape

In [ ]:
#sess = tf.Session()
action = tf.placeholder(tf.float32, [None, 2])
action_2 = tf.placeholder(tf.float32, [None, 2])
state = tf.placeholder(tf.float64, [None, 14])

#state_1 = tf.placeholder(tf.float32, [None, 14])
action_grads = tf.gradients(critic.output, action) #GRADIENTS for policy update
#sess.run(tf.global_variables_initializer())






In [ ]:
#sess = tf.Session()
#sess.run(tf.global_variables_initializer())

def grads (model, states, actions):
    grad = action_grads, feed_dict={
            state: states,
            action: actions
    }
    return grad

grads(critic, test, test2)

In [ ]:
#critic.get_weights()
with tf.Session() as sess:
    print sess.run(action_grads,feed_dict={
    state: test, 
    action: test2})
#critic.get_weights()
#print temp
#sess.close()

In [ ]:
import tensorflow as tf

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a+b

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print sess.run(c,feed_dict={a:1.0, b:1.0})
sess.close()
    

In [ ]:

a = tf.constant(0.)
b = 2 * a
g = tf.gradients(a + b, [a, b], stop_gradients=[a, b])
print g

In [ ]:
actor.output_shape

In [ ]:
memory.append((test, test2))

In [ ]:
len(memory)

In [ ]:
test3 = np.asarray([ 2.76205611,  2.06204796,  1.59506261,  1.24448633,  1.11614728,
        1.05906844,  1.12388086,  1.14259541,  1.03247726,  1.01678181,
        0.        ,  0.        , -2.42      ,  2.35      ])

In [ ]:
memory.append((test, test2, test3, blank))

In [ ]:
t = np.asarray(memory[0])
t2 = np.asarray(memory[2])
t3 = np.asarray(memory[3])

In [ ]:
t3 = t3.reshape(1, len(t3))

In [ ]:
memory[1][1]

In [ ]:
l = np.empty(0)

In [ ]:
states = np.empty(([0,14]), dtype=np.float64)
print states.shape
print np.asarray(memory[1][2]).shape
states = np.append(states, np.asarray(memory[1][2]).reshape(1, 14), axis = 0)

In [ ]:
blank2 = np.append(blank2, blank2, axis = 0)
print blank2.shape
test2 = np.append(test2, test2, axis = 0)
print test2.shape
test = np.append(test, test, axis = 0)
print test.shape


In [ ]:
t = []
t.append(True)
t.append(False)

In [ ]:
loss = 0

In [ ]:
tes= critic.train_on_batch([test, test2], blank2) 

In [ ]:
tes

In [ ]:
class OrnsteinUhlenbeckActionNoise:
    def __init__(self, mu, sigma=0.3, theta=.15, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
                self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        x[0] = abs(x[0])
        self.x_prev = x
        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

    def __repr__(self):
        return 'OrnsteinUhlenbeckActionNoise(mu={}, sigma={})'.format(self.mu, self.sigma)

In [ ]:
actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(2))

In [ ]:
for x in range(200):
    print actor_noise()

In [ ]:
actor_noise()/10